# Interactive figure ligament attachment locations anterior and posterior cruciate ligaments
3D figures showing the ligament attachment locations of the ACL and PCL ligaments on the mean SSM shape of the femur and tibia.
Interactive figure for paper:
Voskuijl, T., Wesseling, M., Pennings, M., Piscaer, T., Hanff, D., Meuffels, D.E. "The adaption of anterior and posterior cruciate ligament attachment sites to the variance of three dimensional bony knee shapes". Submitted to 

Install required packages

In [1]:
#! pip install vtk
#! pip install trimesh
#! pip install seaborn
#! pip install pyvista
#! pip install pythreejs

Import required libraries

In [2]:
import os
import vtk
import trimesh
import numpy as np
import seaborn as sns
import pyvista as pv

Function to create pointcloud that represents attachment regions

In [3]:
def create_pointcloud_polydata(points, colors=None, seg=None):

    vpoints = vtk.vtkPoints()
    vpoints.SetNumberOfPoints(points.shape[0])
    for i in range(points.shape[0]):
        vpoints.SetPoint(i, points[i])

    vpoly = vtk.vtkPolyData()
    vpoly.SetPoints(vpoints)
    rgb_col = []
    if not colors is None:
        if seg == 'femur':
            max_val=8
            color[112:len(color)] = (color[112:len(color)]/max_val)*10
        vcolors = vtk.vtkUnsignedCharArray()
        vcolors.SetNumberOfComponents(3)
        vcolors.SetName("Colors")
        vcolors.SetNumberOfTuples(points.shape[0])
        rgb_col = []
        for i in range(points.shape[0]):
            c = sns.color_palette("viridis_r", n_colors=101, as_cmap=False)
            vcolors.SetTuple3(i, c[int(colors[i] *10)][0]*255, c[int(colors[i] *10)][1]*255, c[int(colors[i] *10)][2]*255)
            rgb_col.append([c[int(colors[i] *10)][0] * 255, c[int(colors[i] *10)][1] * 255, c[int(colors[i] *10)][2] * 255])
        vpoly.GetPointData().SetScalars(vcolors)

    vcells = vtk.vtkCellArray()

    for i in range(points.shape[0]):
        vcells.InsertNextCell(1)
        vcells.InsertCellPoint(i)

    vpoly.SetVerts(vcells)

    return vpoly, rgb_col

Function to load STL file

In [4]:
def load_stl(filename):
    reader = vtk.vtkSTLReader()
    reader.SetFileName(filename)

    mapper = vtk.vtkPolyDataMapper()
    if vtk.VTK_MAJOR_VERSION <= 5:
        mapper.SetInput(reader.GetOutput())
    else:
        mapper.SetInputConnection(reader.GetOutputPort())

    actor = vtk.vtkActor()
    actor.SetMapper(mapper)

    return actor

### Femur attachments

Define variables

In [5]:
segment = 'femur'
center_femur = np.concatenate((np.arange(112),np.arange(341-263)+263))  # PCL + ACL
center = center_femur

Path to bone files

In [6]:
path = os.path.join(r'./data/' + segment)

Load mean SSM and ligament attachment locations

In [7]:
points_lig = trimesh.load_mesh(path + '\meanshape_ligs_color.xyz')
color = np.loadtxt(path + r'\meanshape_ligs_color.xyz')[:, 3]

points_lig = points_lig[center]
color = color[center]

point_cloud_lig, rgb_col = create_pointcloud_polydata(points_lig, colors=color, seg=segment)
bone_actor = load_stl(path + '/mean_shape.stl')
bone_actor.GetProperty().SetOpacity(1.0)

Create actors

In [8]:
bone_actor.GetProperty().SetColor(0.89, 0.85, 0.79)
mapper2 = vtk.vtkPolyDataMapper()
mapper2.SetInputData(point_cloud_lig)
actor2 = vtk.vtkActor()
actor2.SetMapper(mapper2)
actor2.GetProperty().SetColor(1, 0, 0)
actor2.GetProperty().SetPointSize(7.5)

Set colors for ligament attachment points depending on the number of specimens in which each point was identified as attachment region

In [9]:
c = sns.color_palette("viridis_r", n_colors=101, as_cmap=False)
lut = vtk.vtkLookupTable()
lut.SetNumberOfColors(11)
lut.SetTableRange(1, 11)
for j in range(0,11):
    lut.SetTableValue(int(j*1), c[j*10][0], c[j*10][1], c[j*10][2])

Create legend

In [10]:
legend = vtk.vtkScalarBarActor()
labelFormat = vtk.vtkTextProperty()
labelFormat.SetFontSize(16)
titleFormat = vtk.vtkTextProperty()
titleFormat.SetFontSize(8)
legend.SetLabelTextProperty(labelFormat)

legend.SetNumberOfLabels(11)
lut.SetTableRange(0, 100)
legend.SetLookupTable(lut)

legend.SetTitle("% of specimens \n")
legend.SetLabelFormat("%1.0f")
legend.SetUnconstrainedFontSize(1)

text_prop_cb = legend.GetLabelTextProperty()
text_prop_cb.SetFontFamilyAsString('Arial')
text_prop_cb.SetFontFamilyToArial()
text_prop_cb.SetColor(0,0,0)
text_prop_cb.ShadowOff()
legend.SetLabelTextProperty(text_prop_cb)
legend.SetMaximumWidthInPixels(75)
legend.SetMaximumHeightInPixels(300)
legend.SetTitleTextProperty(text_prop_cb)
legend.SetPosition(0.85,0.5)

Visualize bone and attachment locations

In [11]:
plotter = pv.Plotter(window_size=(900, 900),notebook=True)

# bla=pv.PolyData(point_cloud_lig)
# bla.plot()

plotter.background_color = 'w'
#plotter.enable_anti_aliasing()
plotter.add_actor(bone_actor)
plotter.add_mesh(point_cloud_lig, show_scalar_bar=False)
plotter.add_actor(legend)

pv.set_plot_theme("document")
plotter.show()

ViewInteractiveWidget(height=900, layout=Layout(height='auto', width='100%'), width=900)

In [13]:
spheres=[]
plotter = pv.Plotter()
for i in range(0,len(points_lig)):
    spheres.append(pv.Sphere(center=points_lig[i], radius=0.25))
    cols = np.tile(rgb_col[i], (spheres[i].number_of_points,1))
    spheres[i]["colors"] = cols
    plotter.add_mesh(spheres[i])

plotter.add_actor(bone_actor)
plotter.add_actor(legend)
pv.set_plot_theme("document")
plotter.show()  # show the two spheres from two PolyData

plotter.export_html(r"C:\Users\mariskawesseli\Documents\GitLab\2022_JCWMSK_tutorials\SSMfemur.html")

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [14]:
# plotter = pv.Plotter(window_size=(900, 900),notebook=True)
# mesh= pv.read(r"C:\Users\mariskawesseli\Documents\GitLab\femur_lig_ply_col.ply")
# scalars = mesh['RGBA']
# plotter.add_actor(bone_actor)
# plotter.add_mesh(mesh, show_scalar_bar=False, scalars=scalars[:,0:3])
# plotter.add_actor(legend)
# pv.set_plot_theme("document")
# plotter.show()

# plotter.export_html(r"C:\Users\mariskawesseli\Documents\GitLab\2022_JCWMSK_tutorials\SSMfemur.html")

### Tibia

Define variables

In [15]:
segment = 'tibia'
center_tibia = np.concatenate((np.arange(131),np.arange(470-341)+341))  # PCL + ACL
center = center_tibia

Path to bone files

In [16]:
path = os.path.join(r'./data/' + segment)

Load mean SSM and ligament attachment locations

In [17]:
points_lig = trimesh.load_mesh(path + '\meanshape_ligs_color.xyz')
color = np.loadtxt(path + r'\meanshape_ligs_color.xyz')[:, 3]

points_lig = points_lig[center]
color = color[center]

point_cloud_lig, rgb_col = create_pointcloud_polydata(points_lig, colors=color, seg=segment)
bone_actor = load_stl(path + '/mean_shape.stl')
bone_actor.GetProperty().SetOpacity(1.0)

Create actors

In [18]:
bone_actor.GetProperty().SetColor(0.89, 0.85, 0.79)
mapper2 = vtk.vtkPolyDataMapper()
mapper2.SetInputData(point_cloud_lig)
actor2 = vtk.vtkActor()
actor2.SetMapper(mapper2)
actor2.GetProperty().SetColor(1, 0, 0)
actor2.GetProperty().SetPointSize(7.5)

Visualize bone and attachment locations

In [19]:
plotter = pv.Plotter(window_size=(900, 900),notebook=True)

plotter.background_color = 'w'
plotter.enable_anti_aliasing()
plotter.add_actor(bone_actor)
plotter.add_mesh(point_cloud_lig, show_scalar_bar=False)
plotter.add_actor(legend)

pv.set_plot_theme("document")

plotter.show()

ViewInteractiveWidget(height=900, layout=Layout(height='auto', width='100%'), width=900)

In [20]:
spheres=[]
plotter = pv.Plotter()
for i in range(0,len(points_lig)):
    spheres.append(pv.Sphere(center=points_lig[i], radius=0.25))
    cols = np.tile(rgb_col[i], (spheres[i].number_of_points,1))
    spheres[i]["colors"] = cols
    plotter.add_mesh(spheres[i])

plotter.add_actor(bone_actor)
plotter.add_actor(legend)
pv.set_plot_theme("document")
plotter.show()  # show the two spheres from two PolyData

plotter.export_html(r"C:\Users\mariskawesseli\Documents\GitLab\2022_JCWMSK_tutorials\SSMtibia.html")

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [21]:
# plotter = pv.Plotter(window_size=(900, 900),notebook=True)
# mesh= pv.read(r"C:\Users\mariskawesseli\Documents\GitLab\tibia_lig_ply_col.ply")
# scalars = mesh['RGBA']
# plotter.add_actor(bone_actor)
# plotter.add_mesh(mesh, show_scalar_bar=False, scalars=scalars[:,0:3])
# plotter.add_actor(legend)
# pv.set_plot_theme("document")
# plotter.show()

# plotter.export_html(r"C:\Users\mariskawesseli\Documents\GitLab\2022_JCWMSK_tutorials\SSMtibia.html")